# Big Data – Notebook demonstracyjny

Ten notebook prezentuje kolejne kroki ("kolejne krokki") analizy oraz pipeline: MapReduce (lokalna symulacja), eksploracja, przygotowanie cech, model bazowy i ewaluacja. Sekcje zgodne z listą kroków w zadaniu.

## 1. Load Data
Wczytujemy przykładowy plik transakcji CSV z folderu `data/`.

In [ ]:
import pandas as pd
import os
DATA_PATH = os.path.join('..', 'data', 'transactions_sample.csv')

df = pd.read_csv(DATA_PATH)
print('Shape:', df.shape)
print(df.head())

## 2. Validate Raw Schema
Sprawdzamy obecność i typy kolumn oraz podstawowe reguły domenowe.

In [ ]:
expected_cols = {'transaction_id','user_id','amount','category','timestamp'}
assert expected_cols.issubset(df.columns), f"Brak kolumn: {expected_cols - set(df.columns)}"
assert df['amount'].dtype in (float, int, 'float64', 'int64', 'int32', 'float32'), 'Kolumna amount powinna być numeryczna'
negatives = df[df['amount'] < 0]
assert negatives.empty, f"Wykryto ujemne kwoty: {len(negatives)}"
print('Walidacja surowych danych OK.')

## 3. Data Cleaning & Missing Values
Analizujemy brakujące wartości i stosujemy proste uzupełnienia.

In [ ]:
null_counts = df.isna().sum()
print(null_counts)
if 'category' in df.columns:
    df['category'] = df['category'].fillna('UNKNOWN')
if 'amount' in df.columns:
    df['amount'] = df['amount'].fillna(df['amount'].median())
print('Po czyszczeniu nulli:', df.isna().sum())

## 4. Exploratory Profiling
Statystyki opisowe, rozkład i korelacje.

In [ ]:
print(df['amount'].describe())
import seaborn as sns, matplotlib.pyplot as plt
sns.set_theme()
plt.figure(figsize=(6,4))
sns.histplot(df['amount'], bins=8, kde=True)
plt.title('Rozkład amount')
plt.show()

num_cols = [c for c in df.columns if df[c].dtype != 'object']
if num_cols:
    corr = df[num_cols].corr()
    print(corr)


## 5. Feature Engineering
Tworzymy nowe cechy oraz kodowania.

In [ ]:
import numpy as np
if 'amount' in df.columns:
    df['log_amount'] = np.log1p(df['amount'])
if 'category' in df.columns:
    categories = sorted(df['category'].unique())
    cat_to_idx = {c:i for i,c in enumerate(categories)}
    df['category_idx'] = df['category'].map(cat_to_idx)

print(df[['amount','log_amount','category','category_idx']].head())

## 6. Train / Validation Split
Dzielimy dane na zbiór treningowy i walidacyjny.

In [ ]:
from sklearn.model_selection import train_test_split
features = df[['user_id','category_idx']]
labels = df['amount']
X_train, X_val, y_train, y_val = train_test_split(features, labels, test_size=0.3, random_state=42)
print(X_train.shape, X_val.shape)

## 7. Baseline Model
Tworzymy prosty model regresyjny (DummyRegressor) jako punkt odniesienia.

In [ ]:
from sklearn.dummy import DummyRegressor
baseline = DummyRegressor(strategy='mean')
baseline.fit(X_train, y_train)
y_pred_base = baseline.predict(X_val)
from sklearn.metrics import mean_absolute_error, mean_squared_error
print('Baseline MAE:', mean_absolute_error(y_val, y_pred_base))
print('Baseline RMSE:', mean_squared_error(y_val, y_pred_base, squared=False))

## 8. Model Improvement Loop
Dodajemy bardziej zaawansowany model i prostą walidację krzyżową.

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
rf = RandomForestRegressor(n_estimators=100, random_state=42)
cv_scores = cross_val_score(rf, features, labels, cv=3, scoring='neg_mean_absolute_error')
print('CV MAE (neg):', cv_scores)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_val)
print('RF MAE:', mean_absolute_error(y_val, y_pred_rf))
print('RF RMSE:', mean_squared_error(y_val, y_pred_rf, squared=False))

## 9. Evaluation Metrics & Error Analysis
Analiza jakości i błędów modelu.

In [ ]:
import numpy as np
errors = y_val - y_pred_rf
print('Średni błąd:', errors.mean())
print('Top 3 największe dodatnie błędy (niedoszacowanie):')
print(errors.nlargest(3))
print('Top 3 największe ujemne błędy (przeszacowanie):')
print(errors.nsmallest(3))

## 10. Persist Model & Artifacts
Zapisujemy wytrenowany model oraz metadane.

In [ ]:
import joblib, json
os.makedirs('..\\output', exist_ok=True)
model_path = '..\\output\\rf_model.joblib'
joblib.dump(rf, model_path)
meta = {"features": list(features.columns), "target": "amount"}
with open('..\\output\\model_meta.json','w',encoding='utf-8') as f:
    json.dump(meta, f, ensure_ascii=False, indent=2)
print('Zapisano model oraz metadane.')

## 11. Batch Inference Demo
Ładujemy zapisany model i wykonujemy predykcję na próbce walidacyjnej.

In [ ]:
loaded = joblib.load(model_path)
val_sample = X_val.head(5)
print('Wejście batch:', val_sample)
print('Predykcje:', loaded.predict(val_sample))

## 12. Next Action Checklist
- Analiza ważności cech
- Monitoring driftu danych
- Automatyzacja pipeline (CI/CD)
- Parametryzacja hiperparametrów (Grid/Random/Bayesian Search)
- Integracja z systemem produkcyjnym (API / batch)

_Notebook zakończony._